In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping


In [46]:
dataset = pd.read_csv('./jena_climate_2009_2016.csv')
total_null = dataset.isnull().sum().sort_values(ascending=False)
dataset = dataset[:20000]
newset = dataset[['Date Time','T (degC)']]
print("")
print("Total Record:")
print(dataset.shape[0])
print("\n")
print("is nan : \n",newset.isna().sum())
print("\n")
print("is null : \n",newset.isnull().sum())
newset


Total Record:
20000


is nan : 
 Date Time    0
T (degC)     0
dtype: int64


is null : 
 Date Time    0
T (degC)     0
dtype: int64


,Date Time,T (degC)
0,01.01.2009 00:10:00,-8.02
1,01.01.2009 00:20:00,-8.41
2,01.01.2009 00:30:00,-8.51
3,01.01.2009 00:40:00,-8.31
4,01.01.2009 00:50:00,-8.27
...,...,...
19995,19.05.2009 20:40:00,16.51
19996,19.05.2009 20:50:00,16.19
19997,19.05.2009 21:00:00,15.88
19998,19.05.2009 21:10:00,16.05


In [154]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    # series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

In [128]:
scaler = MinMaxScaler()

dates = newset['Date Time'].values
temp  = newset['T (degC)'].values

In [165]:
early_stopping = EarlyStopping(
    monitor='mae',
    patience=10,
    mode='min',
    min_delta=0.1,
    verbose=1
)

In [166]:
temp_train, temp_test, dates_train, dates_test = train_test_split(
    temp, dates, test_size=0.2, shuffle=False)

# Apply Min-Max Scaling to 'T (degC)'
scaler = MinMaxScaler()
temp_scaled = scaler.fit_transform(temp_train.reshape(-1, 1))

train_set = windowed_dataset(
    temp_scaled, window_size=60, batch_size=100, shuffle_buffer=1000)
model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64),
        merge_mode='concat',
    ),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(1),
])

optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.2, momentum=0.5)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set, epochs=100,callbacks=[early_stopping])

Epoch 1/100
160/160 [==============================] - 6s 29ms/step - loss: 0.0024 - mae: 0.0475
Epoch 2/100
160/160 [==============================] - 5s 29ms/step - loss: 3.8132e-04 - mae: 0.0204
Epoch 3/100
160/160 [==============================] - 5s 29ms/step - loss: 2.0960e-04 - mae: 0.0146
Epoch 4/100
160/160 [==============================] - 5s 29ms/step - loss: 1.7467e-04 - mae: 0.0131
Epoch 5/100
160/160 [==============================] - 5s 30ms/step - loss: 1.5875e-04 - mae: 0.0125
Epoch 6/100
160/160 [==============================] - 5s 30ms/step - loss: 1.4971e-04 - mae: 0.0122
Epoch 7/100
160/160 [==============================] - 5s 29ms/step - loss: 1.3956e-04 - mae: 0.0116
Epoch 8/100
160/160 [==============================] - 5s 29ms/step - loss: 1.3433e-04 - mae: 0.0114
Epoch 9/100
160/160 [==============================] - 5s 29ms/step - loss: 1.2847e-04 - mae: 0.0111
Epoch 10/100
160/160 [==============================] - 5s 29ms/step - loss: 1.1850e-04 - mae: 